## Make ADS damage images

In `clean_ads_polygons.ipynb`, we ingested the ADS polygon dataset into Earth Engine. Now, we need to convert these to annual images so that we can add them to our TFRecordDataset.

In [1]:
import ee
import geemap

ee.Initialize()

import os
import datetime

if "notebooks" in os.getcwd():
    os.chdir("..")
    print("Changed working dir to", os.getcwd())

Changed working dir to G:\Other computers\My Laptop\UW\Classes\ESS521\project


## Part 1: damage polygons

Get the asset, filter to an arbitrary year, and map by severity.

In [2]:
ads = ee.FeatureCollection("projects/forest-lst/assets/damage_polygons")
ads_18 = ads.filter(ee.Filter.equals("YEAR", 2018))
print(ads_18.aggregate_histogram("SEVERITY").getInfo())

{'1.0': 287, '2.0': 1154, '3.0': 562, '4.0': 194, '5.0': 22}


In [3]:
severity_codes = [1.0, 2.0, 3.0, 4.0, 5.0]
colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33','#b30000']
labels = ["Very Light", "Light", "Moderate", "Severe", "Very Severe"]

styled_fc = geemap.ee_vector_style(
    ads_18, column='SEVERITY', labels=severity_codes, fillColor=colors, color='000000'
)

Map = geemap.Map()
Map.add_basemap("HYBRID")

Map.addLayer(styled_fc, {}, 'Damage polygons')
Map.add_legend(title='Damage severity', labels=labels, colors=colors)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Rasterize using the MODIS projection, color pixels by severity. If two polys overlap, take the maximum severity.

In [4]:
mod_proj = ee.ImageCollection("MODIS/061/MYD11A1").first().projection()
# Force scale
ads_18_raster = ads_18.reduceToImage(["SEVERITY"], ee.Reducer.max()).rename("SEVERITY").reproject(mod_proj, None, 1000)

In [5]:
# Verify that it worked
img_vis = {
    "min": 1,
    "max": 5,
    "palette": colors
}

Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.addLayer(ads_18_raster, img_vis, "Damage raster")
Map.addLayer(styled_fc, {"color": "#000000", "lineWidth": 2}, 'Damage polygons')
Map.add_legend(title='Damage severity', labels=labels, colors=colors)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Make an image for each year, export it to an image collection.

In [6]:
ca = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("NAME", "California")).first();

def make_ads_damage_image(year, out_collection):
    # Filter to the given year
    ads_filter = ads.filter(ee.Filter.equals("YEAR", year))
    
    # Bail out if this year has no severity data
    if not len(ads_filter.aggregate_array("SEVERITY").getInfo()):
        print(year, "has no severity data")
        return None
    
    ads_raster = ads_filter.reduceToImage(["SEVERITY"], ee.Reducer.max()).rename("SEVERITY").reproject(mod_proj, None, 1000)

    # Set timekeeping properties
    epoch_start = datetime.datetime(year, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc)
    epoch_end   = datetime.datetime(year+1, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc) - datetime.timedelta(milliseconds=1)

    ads_raster = ads_raster.set({
        "system:time_start": epoch_start.timestamp() * 1000,
        "system:time_end": epoch_end.timestamp() * 1000
    })
    
    # Return export task
    fname = "ads_damage_"+str(year)
    asset = "/".join([out_collection, fname])
    
    return ee.batch.Export.image.toAsset(
        ads_raster, description=fname, assetId=asset,
        # N.b. you have to provide a geometry
        region=ca.geometry(), scale=1000, crs=mod_proj
    )

In [7]:
make_ads_damage_image(2020, "projects/forest-lst/assets/damage_img")

<Task Type.EXPORT_IMAGE: ads_damage_2020 (State.UNSUBMITTED)>

In [8]:
# Create destination image collection
collection = "projects/forest-lst/assets/damage_img"
os.system("earthengine create collection {}".format(collection))

0

In [9]:
# Make images for all years of data
years = list(map(int, ads.aggregate_histogram("YEAR").getInfo().keys()))
print(years)

[1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [10]:
tasks = map(lambda x: make_ads_damage_image(x, collection), years)

In [11]:
for t in tasks:
    if t is not None:
        t.start()

1997 has no severity data
1998 has no severity data
1999 has no severity data
2000 has no severity data
2001 has no severity data
2002 has no severity data
2003 has no severity data
2004 has no severity data
2005 has no severity data
2006 has no severity data
2007 has no severity data
2008 has no severity data
2009 has no severity data
2010 has no severity data
2011 has no severity data
2012 has no severity data


## Part 2: survey polygons

More or less doing the same thing here except we set a constant (unmasked!) value of zero for the surveys. This corresponds to a mortality event with zero severity.

In [22]:
surveys = ee.FeatureCollection("projects/forest-lst/assets/survey_polygons")

def make_ads_survey_image(year, out_collection):
    # Grab the polygon
    feature = surveys.filter(ee.Filter.equals("YEAR", year))

    # Add a dummy property
    feature = feature.map(lambda f: f.set("dummy", 0))

    # Rasterize, keep output band as severity for convenience with the damage polygons
    img = feature.reduceToImage(["dummy"], ee.Reducer.first()).rename("SEVERITY").reproject(mod_proj, None, 1000)

    # Set timekeeping properties
    epoch_start = datetime.datetime(year, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc)
    epoch_end   = datetime.datetime(year+1, 1, 1, 0, 0, 0, 
                                    tzinfo=datetime.timezone.utc) - datetime.timedelta(milliseconds=1)

    img = img.set({
        "system:time_start": epoch_start.timestamp() * 1000,
        "system:time_end": epoch_end.timestamp() * 1000
    })

    # Return export task
    fname = "ads_survey_"+str(year)
    asset = "/".join([out_collection, fname])
    
    return ee.batch.Export.image.toAsset(
        img, description=fname, assetId=asset,
        # N.b. you have to provide a geometry
        region=ca.geometry(), scale=1000, crs=mod_proj
    )

In [20]:
survey_collection = "projects/forest-lst/assets/survey_img"
os.system("earthengine create collection {}".format(survey_collection))

0

In [13]:
years = list(map(int, surveys.aggregate_histogram("YEAR").getInfo().keys()))
print(years)

[1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [23]:
tasks = map(lambda x: make_ads_survey_image(x, survey_collection), years)
for t in tasks: t.start()